# Validate

This is a notebook for finding the optimum parameters to train a GPT2 PyTorch language model on Hansard text. This should be ran with a GPU in Google Colab.

To ensure you are running with a GPU go to Runtime -> Change Runtime Type, and ensure GPU is selected as Hardware accelerator

Clones the github repo and downloads required module (HuggingFace transformers)

In [ ]:
!git clone https://github.com/CallumRai/Hansard/
!pip install transformers

Imports required classes and changes working directory

In [ ]:
import os
os.chdir("Hansard")

from hansard import Hansard, Corpus, DataLoader, Trainer
import torch

Downloads hansard between two dates as ```.html```, and converts into ```.json```

The start and end dates can be changed, in YYYY-MM-DD form

In [ ]:
print("Downloading and extracting hansard ...")

s_date = "2019-01-01"
e_date = "2020-06-01"

hansard = Hansard(s_date, e_date)
hansard.download()
hansard.extract()

Creates a corpus of utterances from hansard

The date range must have been previously downloaded, however, can be different to the previous.

In [ ]:
print("Creating corpus ...")

s_date = "2019-01-01"
e_date = "2020-06-01"

corpus = Corpus(s_date, e_date)
corpus.full()

Creates dataloaders for training and validating model

```train_prop``` can be modified to change the proportion of data used to train the model upon

In [ ]:
print("Creating dataloaders ...")

dataloader_path = "hansard/data/dataloaders/"
train_path = dataloader_path + 'train_loader_debug.pth'
val_path = dataloader_path + 'val_loader_debug.pth'

train_prop = 0.75

# If train or val loaders files do not exist make them
if not(os.path.isfile(train_path) or os.path.isfile(val_path)):

    loader_class = DataLoader(s_date, e_date)
    train_loader, val_loader = loader_class.debug(train_prop, 1)

    torch.save(train_loader, train_path)
    torch.save(val_loader, val_path)

train_loader = torch.load(train_path)
val_loader = torch.load(val_path)

Trains the model and prints training and validation loss

```epochs```, ```lr``` (learning rate), ```warmup_steps``` can be changed.

Note: The training step currently takes upwards of 2 hours to run

In [ ]:
trainer = Trainer(train_loader, val_loader)

epochs = 5
lr = 2e-5
warmup_steps = 100

print("Training ...\n")
trainer.debug(epochs, lr, warmup_steps)